# Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- `population` - The population as of 2015.
- `population_growth` - The annual population growth rate, as a percentage.
- `area` - The total land and water area.


In [50]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview of the Data

In [51]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary statistics



Here are the descriptions for some of the columns:

- `name` - The name of the country.
- `area` - The total land and sea area of the country.
- `population` - The country's population.
- `population_growth` - The country's population growth as a percentage.
- `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
- `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
- `area`- The country's total area (both land and water).
- `area_land` - The country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
- `area_water` - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.


In [52]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the summary statistics we see that there is a country in which is reported with population 0.0 and a country which is reported with population 7256490011, which is over 7 billion. This requires further inverstigation.

# Exploring outliers

In [53]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The country corresponding to the value 0 for the population, is Antartica. According to the [CIA Factbook](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html) in fact, there are no inhabitants in Antartica. 

In [54]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The table also contains a row with the world entire population, which explains the value of 7.2 billion for the max population we found.

We recompute the summary statistics excluding the row for the whole world. 

# Summary Statistics Revisited

In [55]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


After excluding the row with the world's population, we see that there is a country with a population of over 1.3 billion.

# Exploring Average Population and Area

We now look at the density of population in the coutries, meaning the ratio between the population and the size of the country.
We start by looking at the average values of the populationand and of country size. We will exclude again the row of the world values.

In [56]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


The average population is 32 million and the average country size is 555 thousand square kilometers.

# Finding Densely Populated Countries

In order to find densely populated coutry, we can look at those countries having a population larger than the average, and a country size smaller than the average.

In [57]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


We have found 7 countries which have a population larger than the average in the dataset and an area smaller than the average. We can therefore consider these countries as some of the most densely populated in our data.

# Additional findings

- What country has the most people? What country has the highest growth rate?

In [58]:
%%sql
SELECT name as Country, MAX(population) AS Population
  FROM facts
  WHERE name <> 'World';

Done.


Country,Population
China,1367485388


In [59]:
%%sql
SELECT name as Country, MAX(population_growth) AS Growth_Rate
  FROM facts;

Done.


Country,Growth_Rate
South Sudan,4.02


China has the largest population with over 1.3 billion and South Sudan has the highest growth rate.

- Which countries have the highest ratios of water to land? Which countries have more water than land?

In [60]:
%%sql
SELECT name, ROUND(area_water/area_land, 2) AS water_to_land
FROM facts
ORDER BY water_to_land DESC
LIMIT 3;

Done.


name,water_to_land
British Indian Ocean Territory,905.0
Virgin Islands,4.0
Afghanistan,0.0


In [61]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


The countries with the biggest ratio of water to land are British Indian Ocean Territory and Virgin Island. These are also the countries which have more water than land.

- Which countries will add the most people to their population next year?

In [69]:
%%sql
SELECT name, population AS current population
FROM facts
WHERE name <> 'World'
LIMIT 10;

(sqlite3.OperationalError) near "population": syntax error
[SQL: SELECT name, population AS current population
FROM facts
WHERE name <> 'World'
LIMIT 10;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [71]:
%%sql
SELECT name, population,
CAST(population*population_growth as integer) as next_year_population,
CAST(population*population_growth-population as integer) as added_population
FROM facts
WHERE name <> 'World'
ORDER BY added_population DESC
LIMIT 1;

Done.


name,population,next_year_population,added_population
India,1251695584,1527068612,275373028


India will add the most people to its population next year, precisely it will add about 275 million people.

- Which countries have a higher death rate than birth rate?

In [85]:
%%sql
SELECT name, death_rate, birth_rate, ROUND(death_rate-birth_rate, 3) AS rate_diff
FROM facts
WHERE rate_diff >=0
ORDER BY rate_diff DESC;

Done.


name,death_rate,birth_rate,rate_diff
Bulgaria,14.44,8.92,5.52
Serbia,13.66,9.08,4.58
Latvia,14.31,10.0,4.31
Lithuania,14.27,10.1,4.17
Ukraine,14.46,10.72,3.74
Hungary,12.73,9.16,3.57
Germany,11.42,8.47,2.95
Slovenia,11.37,8.42,2.95
Romania,11.9,9.14,2.76
Croatia,12.18,9.45,2.73


The above countries have a higher death rate than birth rate. Bulgaria is the country with the boggest ratio. This means that if the trend does not chane, the country population will decrease every year.

- What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

In [87]:
%%sql
SELECT name, population/area AS density_population
FROM facts
ORDER BY population/area DESC
LIMIT 10;

Done.


name,density_population
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


The most densely populated countries are Macau, Monaco, Singapore, Hong Kong, Gaza Strip, Gibraltar, Bahrain, Maldives, Malta and Bermuda.

The list below is very different from the one we looked at before (Finding Densely Populated Countries), because in the previus one we considered only countries with population above average, therefore we mainly considered big countries.